In [2]:
import pandas as pd

dfinit = pd.read_csv("toxity_per_attribute.csv", low_memory=False)
dfinit.columns = dfinit.columns.str.lower()
rows, cols = dfinit.shape
all_init_columns = dfinit.columns
key_columns = ["wiki_id", "toxicity"]
var_columns = [ii for ii in all_init_columns if ii not in key_columns]
dffull = dfinit[var_columns].applymap(lambda v: 1 if v==True else 0)

dffull = dffull.assign(wiki_id=dfinit["wiki_id"], toxicity=dfinit["toxicity"])
dffull = dffull[all_init_columns]


dclasses = {
    "orientation": [
        "heterosexual", "straight", "lesbian", "bisexual", "lgbt", "lgbtq", "gay",
        "queer", "homosexual"
    ],
    "gender": [
        "male", "female", "nonbinary", "transgender", "trans"
    ],
    "race": [
        "white", "american", "canadian", "european", "middle eastern", "chinese", "japanese",
        "asian", "indian", "hispanic", "latino", "latina", "latinx", "mexican", "african", 
        "african american", "black",  
    ],
    "age": [
        "elderly",  "older",  "old",  "middle aged", "millenial", "young", "younger", "teenage",  
    ],
    "religion": [
        "christian", "catholic", "taoist", "protestant", "jewish", "muslim", "buddhist", "sikh",  
    ],
    "disability": [
         "paralyzed", "blind", "deaf",
    ]
    }

for kk in dclasses:
    print(f"\n- protected class: {kk}")
    print("- protected class members:")
    for ii in dclasses[kk]:
        print(f"\t\t+ {ii}")
print("\n")

dffull["row_tot"] = dffull.iloc[:,2:].sum(axis=1)
df = dffull[dffull.row_tot>0].drop("row_tot", axis=1).reset_index(drop=True)

dcondense = {
    "orientation": {
        "hetero_grp": ["heterosexual", "straight"],
        "queer_grp": ["lesbian", "bisexual", "lgbt", "lgbtq", "gay", "queer", "homosexual"],
    },
    "gender": {
        "nonbinary_grp": ["nonbinary", "transgender", "trans"],
    },
    "race": {
        "white_grp": ["white", "american", "canadian", "european"],
        "asian_grp": [ "chinese", "japanese",  "asian", "indian"],
        "hispanic_grp": ["hispanic", "latino", "latina", "latinx", "mexican"],
        "black_grp": ["african", "african american", "black"],
    },
    "age": {
        "old_grp": ["elderly",  "older",  "old"],
        "young_grp": ["young", "younger"],
    },
    "religion": {
        "christian_grp": ["christian", "catholic", "protestant"]
    },  
    "disability": {
        
    }
}

dorder = {
    "orientation": ["hetero_grp", "queer_grp"],
    "gender": ["male", "female", "nonbinary_grp"],
    "race": ["white_grp", "asian_grp", "hispanic_grp", "middle eastern", "black_grp"],
    "religion": ["christian_grp", "jewish", "taoist", "buddhist", "sikh", "muslim"],
    "disability": ["paralyzed", "blind", "deaf"]
    }

# final_grouped_columns = list(
#     itertools.chain.from_iterable([vv for kk,vv in dorder.items()])
#     )

# Collapse categories. 
for category in dorder.keys():
    dcategory = dcondense[category]
    for c_indx, category_grp in enumerate(dorder[category], start=1):
        if category_grp in dcategory.keys():
            category_sub_columns = dcategory[category_grp]
            df[category_grp] = df[category_sub_columns].max(axis=1).replace(1, c_indx)
        else:
            df[category_grp] = df[category_grp].replace(1, c_indx)
            
    
# Collapse df down to 5 categories present in dorder.
for kk in dorder:
    # Outrage machine (take max over values with non-zero value).
    df[kk] = df[dorder[kk]].max(axis=1)

category_columns =  list(dorder.keys())
df5 = df[key_columns + category_columns].reset_index(drop=True)
print(f"df5 number of records: {df5.shape[0]:,.0f}")
print(f"df5 number of columns: {df5.shape[1]:,.0f}") 

df5.head(10)


- protected class: orientation
- protected class members:
		+ heterosexual
		+ straight
		+ lesbian
		+ bisexual
		+ lgbt
		+ lgbtq
		+ gay
		+ queer
		+ homosexual

- protected class: gender
- protected class members:
		+ male
		+ female
		+ nonbinary
		+ transgender
		+ trans

- protected class: race
- protected class members:
		+ white
		+ american
		+ canadian
		+ european
		+ middle eastern
		+ chinese
		+ japanese
		+ asian
		+ indian
		+ hispanic
		+ latino
		+ latina
		+ latinx
		+ mexican
		+ african
		+ african american
		+ black

- protected class: age
- protected class members:
		+ elderly
		+ older
		+ old
		+ middle aged
		+ millenial
		+ young
		+ younger
		+ teenage

- protected class: religion
- protected class members:
		+ christian
		+ catholic
		+ taoist
		+ protestant
		+ jewish
		+ muslim
		+ buddhist
		+ sikh

- protected class: disability
- protected class members:
		+ paralyzed
		+ blind
		+ deaf


df5 number of records: 75,700
df5 number of columns: 7


,wiki_id,toxicity,orientation,gender,race,religion,disability
0,0.0,0.096492,0,0,1,0,0
1,1.0,0.017991,0,3,0,0,0
2,2.0,0.150298,2,0,0,0,0
3,3.0,0.065861,0,0,3,0,0
4,4.0,0.667166,0,0,0,5,0
5,5.0,0.322415,0,0,0,3,0
6,6.0,0.663428,0,0,3,0,0
7,7.0,0.136357,0,3,0,0,0
8,8.0,0.146193,0,0,3,0,0
9,9.0,0.012937,0,0,1,0,0
